In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Loading Library**

In [ ]:
import pandas as pd
import numpy as np
import multiprocessing
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import catboost 
import gc
from time import time
import datetime
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
import seaborn as sns
warnings.simplefilter('ignore')
sns.set()
%matplotlib inline


### 1. Introduction to CatBoost 

Table of Contents

CatBoost documentation says that-

"CatBoost is a high-performance open source library for gradient boosting on decision trees.""

So, CatBoost is an algorithm for gradient boosting on decision trees.
It is a readymade classifier in scikit-learn’s conventions terms that would deal with categorical features automatically.
It can easily integrate with deep learning frameworks like Google’s TensorFlow and Apple’s Core ML.
It can work with diverse data types to help solve a wide range of problems (described later) that businesses face today.
It is developed by Yandex researchers and engineers, and is used for search, recommendation systems, personal assistant, self-driving cars, weather prediction and many other tasks.
Also, it provides best-in-class accuracy.
It is especially powerful in two ways:
1.It yields state-of-the-art results without extensive data training typically required by other machine learning methods, and

2.Provides powerful out-of-the-box support for the more descriptive data formats that accompany many business problems.

“CatBoost” name comes from two words - “Category” and “Boosting”.
It works well with multiple categories of data, such as audio, text, image including historical data.
“Boost” comes from gradient boosting machine learning algorithm as this library is based on gradient boosting library. Gradient boosting is a powerful machine learning algorithm that is widely applied to multiple types of business challenges like fraud detection, recommendation items, forecasting and it performs well also. It can also return very good results with relatively less data, unlike DL models that need to learn from a massive amount of data.
It is in open-source and can be used by anyone.

### 2. Advantages of CatBoost library 
Table of Contents

Advantages of CatBoost library are as follows:-

Performance: CatBoost provides state of the art results and it is competitive with any leading machine learning algorithm on the performance front.
Handling Categorical features automatically: We can use CatBoost without any explicit pre-processing to convert categories into numbers. CatBoost converts categorical values into numbers using various statistics on combinations of categorical features and combinations of categorical and numerical features.
Robust: It reduces the need for extensive hyper-parameter tuning and lower the chances of overfitting also which leads to more generalized models. Although, CatBoost has multiple parameters to tune and it contains parameters like the number of trees, learning rate, regularization, tree depth, fold size, bagging temperature and others.
Easy-to-use: We can use CatBoost from the command line, using an user-friendly API for both Python and R.

### 3. Comparision of CatBoost and other Boosting algorithms 
Table of Contents

We have multiple boosting libraries like XGBoost, H2O and LightGBM and all of these perform well on variety of problems.

CatBoost developer have compared the performance with competitors on standard ML datasets.

This comparision is depicted in the following diagram:

Comparision of CatBoost and other Boosting algorithms

The comparison above shows the log-loss value for test data and it is lowest in the case of CatBoost in most cases. It clearly signifies that CatBoost mostly performs better for both tuned and default models.

In addition to this, CatBoost does not require conversion of data set to any specific format like XGBoost and LightGBM.


### Importing Dataset

In [ ]:
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
sample_submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')


### Head of Data

In [ ]:
train_features.head()

In [ ]:
test_features.head()

In [ ]:
test_features.shape

In [ ]:
train_features.shape

In [ ]:
train_features.dtypes

In [ ]:
train_features.info()

In [ ]:
print("# of common columns : ",len(set(sample_submission.columns.values.tolist()) & set(train_targets_scored.columns.values.tolist())))
print("# of columns in sample submission : ", len(sample_submission.columns.values.tolist()))

In [ ]:
print(train_features.shape)
print(train_targets_scored.shape)

In [ ]:
train_features.describe()

In [ ]:
test_features.describe()

In [ ]:
train_targets_scored.describe()

In [ ]:
train = pd.merge(train_features, train_targets_scored, on='sig_id')

In [ ]:
train.shape,train_features.shape,train_targets_scored.shape

In [ ]:
target_columns = train_targets_scored.columns

In [ ]:
train_targets_scored[\
                     [col for col in train_targets_scored.columns if col not in ['sig_id']]\
                    ].sum(axis=1)\
                    .unique()

In [ ]:
X_train = []
X_train_columns = train.columns

for v in train.values:
    info = v[:876]
    binary = v[876:]
    index = [k for k, i in enumerate(binary) if i==1]
    
    for i in index:
        for k in range(len(binary)):
            if k==i:
#                 binary_transformed = list(copy.copy(binary))
#                 binary_transformed[i] = 0
                X_train.append(list(info) + [X_train_columns[876+k]])

X_train = pd.DataFrame(X_train, columns=train_features.columns.tolist() + ['pred'])


In [ ]:
X_train.shape

In [ ]:
X_train.head()

In [ ]:
X_test = test_features
y_train = X_train[['pred']]

In [ ]:
y_train.value_counts()

In [ ]:
y_train.head()

### Unique value

In [ ]:
len(y_train['pred'].unique().tolist())

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

X_test['pred'] = -1
data = X_train.append(X_test)

for col in ['cp_type','cp_time','cp_dose']:
    data.loc[:, col] = le.fit_transform(data[col])
    
X_train = data[:X_train.shape[0]]
X_test = data[-X_test.shape[0]:]

In [ ]:
mapping = {v:k for k,v in enumerate(sample_submission.iloc[:,1:].columns.tolist())}
# inverse_mapping = {k:v for k,v in enumerate(sample_submission.iloc[:,1:].columns.tolist())}

In [ ]:
y_train['pred'] = y_train['pred'].map(mapping)
y_train

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
cat_features = ['cp_type','cp_time','cp_dose']


In [ ]:
model = CatBoostClassifier(task_type='GPU',iterations = 50000,learning_rate = 0.1)

model.fit(X_train.drop(columns=['sig_id','pred']),
          y_train,
          verbose=5000,
          cat_features=cat_features)

In [ ]:
prediction = model.predict_proba(X_test.drop(columns=['sig_id'], axis=1))

In [ ]:
sample_submission.iloc[:, 1:] = prediction

In [ ]:
sample_submission.to_csv('submission.csv',index=False)